## 1. 環境構築

In [ ]:
# GPU確認
!nvidia-smi

import torch
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Device name: {torch.cuda.get_device_name(0)}")

### 4. Git pull

In [ ]:
!pip install nbstripout
!nbstripout --install

In [ ]:
!git config --global user.email "runpod@example.com"
!git config --global user.name "Runpod User"

In [ ]:
# # プロジェクトディレクトリの設定
# import os

# # ★ここを自分のリポジトリURLに変更★
# GITHUB_REPO = "https://github.com/snoopy0420/atma_22_ca.git"
# PROJECT_NAME = "atma_22_ca"

# # Git clone（初回のみ）
# if not os.path.exists(f"/content/{PROJECT_NAME}"):
#     !git clone {GITHUB_REPO} /content/{PROJECT_NAME}
#     print(f"✅ Cloned {GITHUB_REPO}")
# else:
#     print(f"⚠️ {PROJECT_NAME} already exists. Pulling latest changes...")
#     !cd /content/{PROJECT_NAME} && git pull

# # プロジェクトディレクトリに移動
# %cd /content/{PROJECT_NAME}

In [ ]:
!git pull origin main

## 2. パッケージインストール

In [ ]:
# 必要パッケージのインストール
!pip install -q -r ../requirements.txt

## 6. モジュールインポート

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch

# プロジェクトルートをパスに追加
sys.path.append('/workspace/atma_22_ca/')

from configs.config import *
from src.runner import Runner
from src.model_resnet_knn import ModelResNet50KNN
from src.util import Logger, Submission

print(f"Input dir: {DIR_INPUT}")
print(f"Model dir: {DIR_MODEL}")
print(f"Submission dir: {DIR_SUBMISSIONS}")

## 7. ロガー設定

In [ ]:
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    run_name = model_type
    suffix = '_' + datetime.now().strftime("%Y%m%d%H%M")
    run_name = run_name + suffix
    return run_name

logger.info("Logger initialized on runpod")

## 8. データ読み込み

In [ ]:
df_train = pd.read_csv(FILE_TRAIN_META)
df_test = pd.read_csv(FILE_TEST_META)

# グループ化
df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)


print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")
print(f"\n【CV Strategy】")
print(f"  Group column: 'group' (quarter_session)")
print(f"  Total groups: {df_train['group'].nunique()}")
print(f"  Avg samples per group: {len(df_train) / df_train['group'].nunique():.1f}")

## 9. パラメータ設定（GPU最適化）

### GPU最適化の戦略
NVIDIA A40 (46GB VRAM) を活用した高速化：

**実装済み最適化**
1. **batch_size**: 64 → 256 (4倍)
   - VRAM容量が十分なため大幅増加
   - 1バッチあたりの処理効率が向上
   
2. **num_workers**: 4 → 8 (2倍)
   - CPU並列データローディング
   - GPU待ち時間を削減
   
3. **persistent_workers**: True
   - ワーカープロセスを再利用
   - epoch/fold間のオーバーヘッド削減
   
4. **prefetch_factor**: 2
   - 次のバッチを事前ロード
   - GPU idle時間を最小化
   
5. **non_blocking=True**
   - GPU転送を非同期化
   - CPU-GPU間の並列実行

**期待される高速化**
- 特徴抽出: 約3-4倍高速化
- 全体の学習時間: 20-30分 → 5-10分程度

In [ ]:
memo = "ResNet50 + KNN on runpod"
run_name = get_run_name("resnet50_knn_runpod")
# run_name = "resnet50_knn_runpod_202512180257"

In [ ]:
# モデルパラメータ設定
# ※ 前処理済み画像は自動検出して使用されます（data/raw/input/train_crops/が存在する場合）
params = {
    'model_name': 'resnet50',
    # KNN固有パラメータ
    'k': 5,
    # 閾値設定
    'threshold': 0.5,
    'min2_threshold': 0.3,
    # GPU最適化パラメータ
    'batch_size': batch_size,
    'num_workers': num_workers,
    # キャッシュ設定
    'use_cache': True,
}

# CV設定
cv_setting = {
    'method': 'group',  # 'group' (推奨) or 'stratified_group'
    'group_col': 'group',  # 上で作成したquarter_session列
    'n_splits': 5,
    # 注: GroupKFoldはshuffleパラメータなし（時系列順を保つ）
}

## 11. Runner作成

In [ ]:
# Runnerの作成
runner = Runner(
    run_name=run_name,
    model_cls=ModelResNet50KNN,
    params=params,
    df_train=df_train,
    df_test=df_test,
    cv_setting=cv_setting,
    logger=logger
)

print(f"\nRunner created: {run_name}")
print(f"Model class: {ModelResNet50KNN.__name__}")

## 12. 5-Fold CV学習（GPU高速化）

In [ ]:
%%time
# 5-fold CV学習
runner.train_cv()

# 5-fold 評価

In [ ]:
scores, oof_score = runner.metric_cv()

## 13. テストデータ予測

In [ ]:
%%time
# CV全モデルのアンサンブル予測
pred_test = runner.predict_cv()

In [ ]:
# 予測分布の可視化
plt.figure(figsize=(10, 5))
pred_test['label_id'].value_counts().sort_index().plot(kind='bar')
plt.title('Test Prediction Distribution (KNN - Local)')
plt.xlabel('Label ID')
plt.ylabel('Count')
plt.axhline(y=len(pred_test)/11, color='r', linestyle='--', alpha=0.5, label='Uniform')
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 14. 提出ファイル作成

In [ ]:
Submission.save(pred_test, run_name=run_name, logger=logger)

## 15. kチューニング実験（高速）

In [ ]:
# kの値を変えて予測（類似度キャッシュ再利用で高速）
k_values = [3, 5, 7, 10, 15]
k_results = []

print("\n=== Testing different k values ===")
print("(Using cached similarities for fast tuning)\n")

for k in k_values:
    predictions_list = []
    
    for i_fold in range(runner.n_splits):
        model = runner.load_model_cv(i_fold)
        
        if hasattr(model, 'test_similarities') and model.test_similarities is not None:
            pred = model.predict_with_custom_threshold(
                threshold=params['threshold'],
                min2_threshold=params['min2_threshold'],
                k=k
            )
            predictions_list.append(pred)
    
    if predictions_list:
        predictions_array = np.array(predictions_list)
        final_pred = []
        for i in range(predictions_array.shape[1]):
            values, counts = np.unique(predictions_array[:, i], return_counts=True)
            final_pred.append(values[np.argmax(counts)])
        
        unknown_count = np.sum(np.array(final_pred) == -1)
        unknown_ratio = unknown_count / len(final_pred) * 100
        
        k_results.append({
            'k': k,
            'unknown_count': unknown_count,
            'unknown_ratio': unknown_ratio
        })
        
        print(f"k={k:2d}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

df_k_results = pd.DataFrame(k_results)
print("\n=== k-value tuning results ===")
print(df_k_results)

In [ ]:
# kとunknown比率の関係を可視化
if len(k_results) > 0:
    plt.figure(figsize=(10, 5))
    plt.plot(df_k_results['k'], df_k_results['unknown_ratio'], marker='o', linewidth=2)
    plt.xlabel('k (Number of neighbors)', fontsize=12)
    plt.ylabel('Unknown Ratio (%)', fontsize=12)
    plt.title('KNN: Effect of k on Unknown Predictions (Local)', fontsize=14)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()